In [1]:
import json
from numpyencoder import NumpyEncoder
import sys
sys.path.insert(0, '../../')
from util.evaluator import *

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
def run_cemsp(json_path):
    with open(json_path) as f:
        cemsp_json = json.load(f)

    cemsp_json.keys()
    d = len(cemsp_json['data'][0][0])
    cfs_list = cemsp_json['cf']
    _cfs_list = cemsp_json['cf2']
    diversity_list = []
    diversity2_list = []

    count_diversity_list = []
    count_diversity2_list = []

    for cfs, _cfs in zip(cfs_list, _cfs_list):
        cfs = [item['cf'] for item in cfs]
        cfs = np.reshape(cfs, (-1, d))
        # print(cfs.shape)
        diversity = evaluator.diversity(cfs)
        count_diversity = evaluator.count_diversity(cfs)

        _cfs = [item for item in _cfs]
        _cfs = np.reshape(_cfs, (-1, d))
        print(_cfs.shape)
        diversity2 = evaluator.diversity(_cfs)
        count_diversity2 = evaluator.count_diversity(_cfs)

        diversity_list.append(diversity)
        diversity2_list.append(diversity2)
        count_diversity_list.append(count_diversity)
        count_diversity2_list.append(count_diversity2)

    print(cemsp_json['diversity'])
    print(diversity_list)

    print(cemsp_json['diversity2'])
    print(diversity2_list)


    cemsp_json['diversity'] = diversity_list
    cemsp_json['diversity2'] = diversity2_list
    cemsp_json['count_diversity'] = count_diversity_list
    cemsp_json['count_diversity2'] = count_diversity2_list
    with open(json_path, "w") as f:
        json.dump(cemsp_json, f, cls = NumpyEncoder)

def run(json_path):
    with open(json_path) as f:
        cemsp_json = json.load(f)

    cemsp_json.keys()
    d = len(cemsp_json['data'][0][0])
    cfs_lists = cemsp_json['cf']
    diversity_list = []
    diversity2_list = []

    count_diversity_list = []
    count_diversity2_list = []

    for cfs_list in cfs_lists:

        cfs = [_tmp_result['cf'] for _tmp_result in cfs_list]
        cfs = np.reshape(cfs, (-1, d))
        diversity = evaluator.diversity(cfs)
        count_diversity = evaluator.count_diversity(cfs)

        _cfs = [_tmp_result['cf2'] for _tmp_result in cfs_list]
        _cfs = np.reshape(_cfs, (-1, d))
        diversity2 = evaluator.diversity(_cfs)
        count_diversity2 = evaluator.count_diversity(_cfs)

        diversity_list.append(diversity)
        diversity2_list.append(diversity2)
        count_diversity_list.append(count_diversity)
        count_diversity2_list.append(count_diversity2)

    print(cemsp_json['diversity'])
    print(diversity_list)

    print(cemsp_json['diversity2'])
    print(diversity2_list)

    cemsp_json['diversity'] = diversity_list
    cemsp_json['diversity2'] = diversity2_list
    cemsp_json['count_diversity'] = count_diversity_list
    cemsp_json['count_diversity2'] = count_diversity2_list
    with open(json_path, "w") as f:
        json.dump(cemsp_json, f, cls = NumpyEncoder)

# hepatitis

In [3]:
from hepatitis_update import *

data = pd.read_csv("../../data/Hepatitis/HepatitisC_dataset_processed.csv")
standard_sc = StandardScaler()

X = data.drop(['Category'],axis=1)
y = data["Category"]
train_x, test_x, train_y, test_y = train_test_split(X,y,test_size=0.3,random_state=42,stratify=y)

train_x=standard_sc.fit_transform(train_x).astype(np.float32)
test_x=standard_sc.transform(test_x).astype(np.float32)

train_y = train_y.to_numpy().astype(np.int64)
test_y = test_y.to_numpy().astype(np.int64)

model_path = '../../train/hepatitis/Hepatitis_model_simple.pt'
baseline_model = create_model_from_torch(model_path)
model_path1 = '../../train/hepatitis/Hepatitis_model_simple_v1.pt'
baseline_model1 = create_model_from_torch1(model_path1)

# obtain true positive set of test set
idx = np.where(test_y == 0)[0]
pred_y = np.argmax(baseline_model.predict(test_x), axis = 1)
idx1 = np.where(pred_y == 0)[0]
pred_y_ = np.argmax(baseline_model1.predict(test_x), axis = 1)
idx1_ = np.where(pred_y_ == 0)[0]
tn_idx = set(idx).intersection(idx1)
tn_idx = set(tn_idx).intersection(idx1_)
abnormal_test = test_x[list(tn_idx)]

# obtain true negative set of train set
idx2 = np.where(train_y == 1)[0]
pred_ty = baseline_model.predict(train_x).round()
pred_ty = np.argmax(baseline_model.predict(train_x), axis = 1)
idx3 = np.where(pred_ty == 1)[0]
tp_idx = set(idx2).intersection(idx3)
normal_test = train_x[list(tp_idx)]

# initialize the evaluator
evaluator = Evaluator(train_x, normal_test)

14/14 [==============================] - 0s 923us/step


In [4]:
hepatitis_update_path = 'hepatitis_sns_update.json'
run(hepatitis_update_path)

[0.8431685039902644, 0.019210262688733937, 0.4578612149787101, 0.11044469531564438, 0.14384940322352194, 0.07140411605324104, -1, -1, -1, 0.1864960269985505, -1, 0.6062073405112304, 0.030632713968893555, 0.6105774374641542, -1, 0.12314295599675597, -1, 0.11715493946706168, 0.2760479154056929]
[0.8431685039902644, 0.019210262688733937, 0.4578612149787101, 0.11044469531564438, 0.14384940322352194, 0.07140411605324104, -1, -1, -1, 0.1864960269985505, -1, 0.6062073405112304, 0.030632713968893555, 0.6105774374641542, -1, 0.12314295599675597, -1, 0.11715493946706168, 0.2760479154056929]
[0.2040274039272608, 0.028074460020895753, 0.22448933021694062, 1.2858315988446052, 0.5829537048430262, 0.22841162198887233, -1, -1, -1, 0.1641231846751685, -1, 0.1098755771808344, 0.027391501099994404, 0.7122444795113434, -1, 0.662811398459702, -1, 0.6534181234935872, 0.2503645476512194]
[0.2040274039272608, 0.028074460020895753, 0.22448933021694062, 1.2858315988446052, 0.5829537048430262, 0.2284116219888723

# Syn

In [5]:
from synthetic_update import *

dataset = pd.read_csv("../../data/synthetic/synthetic_data_simple.csv")
dataset = dataset.to_numpy().astype(np.float32)
lens = len(dataset)
train, test = dataset[0:int(lens * 0.7), ], dataset[int(lens*0.7):, ]
train_x, train_y = train[:, 0:4], train[:, 4]
test_x, test_y = test[:, 0:4], test[:, 4]
train_y = train_y.astype(np.int64)
test_y = test_y.astype(np.int64)

model_path = '../../train/synthetic/synthetic_model_simple.pt'
baseline_model = create_model_from_torch(model_path)
model_path1 = '../../train/synthetic/synthetic_model_simple_v1.pt'
baseline_model1 = create_model_from_torch1(model_path1)

# obtain true positive set of test set
idx = np.where(test_y == 0)[0]
pred_y = np.argmax(baseline_model.predict(test_x), axis = 1)
idx1 = np.where(pred_y == 0)[0]
pred_y_ = np.argmax(baseline_model1.predict(test_x), axis = 1)
idx1_ = np.where(pred_y_ == 0)[0]
tn_idx = set(idx).intersection(idx1)
tn_idx = set(tn_idx).intersection(idx1_)
abnormal_test = test_x[list(tn_idx)]

# obtain true negative set of train set
idx2 = np.where(train_y == 1)[0]
pred_ty = baseline_model.predict(train_x).round()
pred_ty = np.argmax(baseline_model.predict(train_x), axis = 1)
idx3 = np.where(pred_ty == 1)[0]
tp_idx = set(idx2).intersection(idx3)
normal_test = train_x[list(tp_idx)]

# initialize the evaluator
evaluator = Evaluator(train_x, normal_test)

438/438 [==============================] - 0s 645us/step


In [6]:
syn_update_path = 'synthetic_sns_update.json'
run(syn_update_path)

[0.8022054967002697, 1.9494585131089412, 2.4119700800043495, 0.5928401069041982, 1.7246814929821692, 1.9079517728411564, 1.5636512792263169, 0.3667410039706738, 1.2484376380580902, 2.00236307466824, 0.7198618967853202, 3.228815179917899, 1.5556641087703678, 1.5357352854786857, 0.763116143451016, 0.23256609513829218, 2.3688739709914213, 0.36744921830764987, 1.6401902752388968, 4.051325232851771, 1.4611629298166733, 0.48348201669081803, 2.8077624765156894, 0.3727896525809579, 4.155194680132536, 0.9981842643633803, 0.6766789257839037, 1.5063359482523804, 1.455066924249572, 0.02482330210010275, 4.853204373863608, 0.17562800423603164, 0.6665472237497561, 1.4974633987393995, 1.2159858233936898, 2.9607417753489944, 0.7588498299397539, 0.6460481839072341, 4.640970367060035, 2.1443049607453846, 0.6862349244508161, 0.8176256761721032, 0.5854009983548139, 0.8662226128629333, 4.608898767399211, 6.926413839730161, 1.0855081169401828, 2.7214199007770845, 1.5665170128941242, 0.6068895838563542, 0.677

# thyroid

In [7]:
from thyroid_update import *

dataset = pd.read_csv("../../data/thyroid/thyroid_dataset.csv")
dataset = dataset.to_numpy().astype(np.float32)
lens = len(dataset)
train, test = dataset[0:int(lens * 0.7), ], dataset[int(lens*0.7):, ]
train_x, train_y = train[:, 0:4], train[:, 4]
test_x, test_y = test[:, 0:4], test[:, 4]

scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)
train_y = train_y.astype(np.int64)
test_y = test_y.astype(np.int64)

model_path = '../../train/thyroid/thyroid_data.pt'
baseline_model = create_model_from_torch(model_path)
model_path1 = '../../train/thyroid/thyroid_data_new.pt'
baseline_model1 = create_model_from_torch1(model_path1)

# obtain true positive set of test set
idx = np.where(test_y == 0)[0]
pred_y = np.argmax(baseline_model.predict(test_x), axis = 1)
idx1 = np.where(pred_y == 0)[0]
pred_y_ = np.argmax(baseline_model1.predict(test_x), axis = 1)
idx1_ = np.where(pred_y_ == 0)[0]
tn_idx = set(idx).intersection(idx1)
tn_idx = set(tn_idx).intersection(idx1_)
abnormal_test = test_x[list(tn_idx)]

# obtain true negative set of train set
idx2 = np.where(train_y == 1)[0]
pred_ty = baseline_model.predict(train_x).round()
pred_ty = np.argmax(baseline_model.predict(train_x), axis = 1)
idx3 = np.where(pred_ty == 1)[0]
tp_idx = set(idx2).intersection(idx3)
normal_test = train_x[list(tp_idx)]

# initialize the evaluator
evaluator = Evaluator(train_x, normal_test)

61/61 [==============================] - 0s 821us/step


In [8]:
thyroid_update_path = 'thyroid_sns_update.json'
run(thyroid_update_path)

[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
